In [1]:
!pip install soynlp

In [2]:
import pandas as pd
import numpy as np

In [3]:
# train_df=pd.read_csv("train_data.csv",engine="python",encoding="utf-8")[:1000]
# test_df=pd.read_csv("test_data.csv",engine="python",encoding="utf-8")[:1000]
# df=pd.concat([train_df,test_df])
df=pd.read_csv("post.csv",engine="python",encoding="utf-8")
df

,Unnamed: 0,insta_id,content,label
0,0,hahajuhye,마이삭 조용히지나가길 기원하며 Rain rain go away,0.0
1,1,hahajuhye,엄마~ 오늘 재하 생일이지? 엄마~ 오늘 재야 생일이디~? 엄마~ 엄ㅁㅏ~ #그래...,0.0
2,3,hahajuhye,조..좋은언니...2,0.0
3,4,hahajuhye,좋은언니♡ #흔들리는너의눈동자 #걱정마엄마여기있다,0.0
4,5,hahajuhye,Look Im serious,0.0
...,...,...,...,...
1341157,1372663,cream_2299,내가 이뽀라하는 👩‍⚖️랑 일본식 선술집에서 나란히 앉자 이밤 즐기기 #모히또 ...,3.0
1341158,1372664,cream_2299,오늘은 계속 달달한게 땡기는 몹쓸 날😵😚 🍠 는 태워뿔고 🍦 은 일보다 급 u턴후 ...,3.0
1341159,1372665,cream_2299,울집 우체통에 선물이 살포시..비염이 오면 오한이 동시타임으로 휩쓸고 가버려 힘드...,3.0
1341160,1372666,cream_2299,오늘은 달콤한 #라떼 #임계무농약 사과 🍎 와작~ #보광리 팅구 ☕️ 숍 #드라이 ...,3.0


In [4]:
# #임의로 이 데이터들로만 다뤄보겠습니다.
# df=pd.concat([df.loc[df["label"]==0][:500],df.loc[df["label"]==1][:500],df.loc[df["label"]==2][:500],df.loc[df["label"]==3][:500]])

In [5]:
insta_id=df["insta_id"].tolist()
content=df["content"].tolist()
label=df["label"].tolist()

In [6]:
df=pd.DataFrame({"insta_id":insta_id,"content":content,"label":label})

In [7]:
def del_escape(text):
    char_ls=list(text)
    reduced_ls=[char for char in char_ls if "{c!r}".format(c=char).replace("'","")==char]
    return "".join(reduced_ls)

In [8]:
data=df.copy()
data=data.dropna()
data["content"]=data["content"].apply(del_escape)
data=data.dropna()

In [9]:
data["content"].to_csv("post_content.csv",index=False)

In [10]:
from soynlp import DoublespaceLineCorpus

corpus_path = "post_content.csv"
sentences = DoublespaceLineCorpus(corpus_path, iter_sent=True)

In [11]:
from soynlp.word import WordExtractor
from soynlp.noun import LRNounExtractor_v2
noun_extractor = LRNounExtractor_v2()

noun_extractor.train(sentences) # list of str or like
nouns = noun_extractor.extract(min_noun_frequency=3000)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 6352947 from 2987868 sents. mem=2.836 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=33867543, mem=20.860 Gb
[Noun Extractor] batch prediction was completed for 1064608 words
[Noun Extractor] checked compounds. discovered 1772500 compounds
[Noun Extractor] postprocessing detaching_features : 698 -> 693
[Noun Extractor] postprocessing ignore_features : 693 -> 639
[Noun Extractor] postprocessing ignore_NJ : 639 -> 622
[Noun Extractor] 622 nouns (1772500 compounds) with min frequency=3000
[Noun Extractor] flushing was done. mem=19.825 Gb                    
[Noun Extractor] 25.26 % eojeols are covered


In [12]:
len(nouns)

622

In [13]:
user_seg_df=data.pivot_table(index="insta_id",values=["label"],aggfunc="median")
user_seg_df=user_seg_df.reset_index()
user_seg_df

,insta_id,label
0,0._.do__,1.0
1,0.____.won,1.0
2,000._.bd,0.0
3,00000_min04,0.0
4,00_12.21,0.0
...,...,...
6723,zzung_e_77,1.0
6724,zzuuuu5,1.0
6725,zzveddd6,1.0
6726,zzz436,0.0


In [14]:
user_text_df=data.pivot_table(index="insta_id",values=["content"],aggfunc="sum")
user_text_df=user_text_df.reset_index()
user_text_df

,insta_id,content
0,0._.do__,약속은 약속이니까✨ ㅋㅋ ㅋ ㅋ 내일도 인증해야지 .. 그래...
1,0.____.won,추억82☆......빨대길이가 왜 다른거야 갑자기 신경쓰여🤨 Photo by @ye...
2,000._.bd,͏s͏i͏m͏p͏l͏e ͏i͏s ͏t͏h͏e ͏b͏e͏s͏t아프로뒷태여신이거 올리래...
3,00000_min04,배구에 다시 반해버렸다Gs 화이팅... 박혜민 선수 화이팅..❤❤Good frien...
4,00_12.21,나 승모근 굉장한 사람이니까 깝치면 다 뒤진다･ﾟ⋆ 🎀 𝒯𝒽𝑒 𝒹𝒶𝓃𝒷𝒾𝓃𝑒 🎀 ...
...,...,...
6723,zzung_e_77,이게 얼마만이여~~~ #됐어요 #돌아왔으니이제됐어요 #너무오래걸려 #걱정해짜나~~~...
6724,zzuuuu5,내 방에는 의자가 없음🙉 그래서 스터디카페에서 하거나 식탁에서 하는 이유지만 요즘은...
6725,zzveddd6,~~😆😊 ⠀⠀⠀⠀⠀⠀⠀ ⠀⠀⠀⠀⠀ ⠀⠀⠀⠀ ⠀⠀⠀⠀ ⠀⠀⠀⠀⠀⠀ ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀...
6726,zzz436,#건강한습관만들기✔️ #산책하기 #건강하게먹기 #근력운동10분 #일찍자고일찍일어...


In [15]:
dic={}

for term in nouns:
    try:
        reduced_term=term.replace("(","").replace(")","")
        dic[reduced_term]=[sum(data.loc[data["label"]==0]["content"].str.contains(reduced_term)),sum(data["label"]==0)-sum(data.loc[data["label"]==0]["content"].str.contains(reduced_term)),
           sum(data.loc[data["label"]!=0]["content"].str.contains(reduced_term)),sum(data["label"]!=0)-sum(data.loc[data["label"]!=0]["content"].str.contains(reduced_term))]
    except:
        print("not ordinary")

dic  

not ordinary
not ordinary
not ordinary


{'#선팔하면맞팔': [5659, 819923, 10119, 500867],
 '[더블마케팅]': [391113, 434469, 209120, 301866],
 '#selfie': [12091, 813491, 7232, 503754],
 '#일상스타그램': [20872, 804710, 11584, 499402],
 '#다이어트식단': [1277, 824305, 2271, 508715],
 '#크리스마스': [5879, 819703, 2429, 508557],
 '#인스타그램': [5010, 820572, 3501, 507485],
 '#공스타그램': [1152, 824430, 11357, 499629],
 '#셀스타그램': [14260, 811322, 6665, 504321],
 '7일~15일': [0, 825582, 4497, 506489],
 '#공스타맞팔': [390, 825192, 3791, 507195],
 '#일상#소통': [7969, 817613, 2306, 508680],
 '#맛스타그램': [15486, 810096, 5461, 505525],
 '#다이어트': [7720, 817862, 13403, 497583],
 '2020년': [2652, 822930, 2872, 508114],
 '크리스마스': [8692, 816890, 3741, 507245],
 '인스타그램': [13205, 812377, 6914, 504072],
 '#육아소통': [23539, 802043, 2995, 507991],
 '#가족여행': [9389, 816193, 1929, 509057],
 '#스타벅스': [3896, 821686, 2138, 508848],
 '카톡아이디': [4793, 820789, 80, 510906],
 '실측사이즈': [5614, 819968, 8, 510978],
 '#데일리룩': [25996, 799586, 7730, 503256],
 '#아들둘맘': [7191, 818391, 806, 510180],
 '#불량소녀': [5073, 

In [16]:
mll_dm={}
mll_im={}
aic_dm={}
aic_im={}
relevance={}
eps=1e-12
for x in dic:
    mll_dm[x]=dic[x][0]*np.log(dic[x][0]+eps)+dic[x][1]*np.log(dic[x][1]+eps)+dic[x][2]*np.log(dic[x][2]+eps)+dic[x][3]*np.log(dic[x][3]+eps)-(len(data))*np.log(len(data))
for x in dic:
    mll_im[x]=sum(data["label"]==0)*np.log(sum(data["label"]==0))+sum(data["label"]!=0)*np.log(sum(data["label"]!=0))+(dic[x][0]+dic[x][2])*np.log(dic[x][0]+dic[x][2]+eps)+(dic[x][1]+dic[x][3])*np.log(dic[x][1]+dic[x][3]+eps)-2*(len(data))*np.log(len(data))    
for x in dic:
    aic_dm[x]=-2*mll_dm[x]+6
for x in dic:
    aic_im[x]=-2*mll_im[x]+4
for x in dic:
    if dic[x][0]/(dic[x][0]+dic[x][2]+eps)>dic[x][1]/(dic[x][1]+dic[x][3]+eps):
        relevance[x]=aic_im[x]-aic_dm[x]+2
    else:
        relevance[x]=aic_dm[x]-aic_im[x]-2

In [17]:
characteristic_term={}
i=0
for x in relevance:
    if relevance[x]>0:
        characteristic_term[i]=x
        i+=1
characteristic_term

{0: '[더블마케팅]',
 1: '#selfie',
 2: '#일상스타그램',
 3: '#크리스마스',
 4: '#셀스타그램',
 5: '#일상#소통',
 6: '#맛스타그램',
 7: '크리스마스',
 8: '인스타그램',
 9: '#육아소통',
 10: '#가족여행',
 11: '#스타벅스',
 12: '카톡아이디',
 13: '실측사이즈',
 14: '#데일리룩',
 15: '#아들둘맘',
 16: '#불량소녀',
 17: '#육아그램',
 18: '#맞팔환영',
 19: '보내주시면',
 20: '#아들바보',
 21: '#오오티디',
 22: '어린이집',
 23: '하루종일',
 24: '#귀요미',
 25: '#데이트',
 26: '#우리집',
 27: '남녀공용',
 28: '#내사랑',
 29: '회원가입',
 30: '무료배송',
 31: '#일요일',
 32: '#토요일',
 33: '#화이팅',
 34: '#물놀이',
 35: '#이벤트',
 36: '#육아맘',
 37: '#스타일',
 38: '#10대',
 39: '#팔로우',
 40: '#고마워',
 41: '다운로드',
 42: '#아들맘',
 43: '#나들이',
 44: '#도치맘',
 45: '9/10',
 46: '#좋아요',
 47: '퍼니그램',
 48: '#제주살',
 49: '가슴단면',
 50: '#셀스타',
 51: '너무너무',
 52: 'size',
 53: '#세젤예',
 54: '#엄마',
 55: '마스크',
 56: '이야기',
 57: '집에서',
 58: '유치원',
 59: '#독서',
 60: '#기록',
 61: '아이들',
 62: '#여름',
 63: '#겨울',
 64: '#사랑',
 65: '#그림',
 66: '#가을',
 67: '#오늘',
 68: '#점심',
 69: '한국인',
 70: '이렇게',
 71: '데일리',
 72: '#아침',
 73: '선생님',
 74: '...',
 75: '#주말',
 76: '인스타',


In [18]:
X=np.array(user_text_df["content"].apply(lambda x:[int(characteristic_term[i] in x) for i in range(len(characteristic_term))]).tolist())
y=np.array((user_seg_df["label"]==0).apply(int))

# X_train=np.array(train_user_text_df["content"].apply(lambda x:[int(characteristic_term[i] in x) for i in range(len(characteristic_term))]).tolist())
# y_train=np.array((train_user_seg_df["label"]==0).apply(int))

In [19]:
# train_df=pd.DataFrame({"insta_id":train_insta_id,"content":train_content,"label":train_label})
# test_df=pd.DataFrame({"insta_id":test_insta_id,"content":test_content,"label":test_label})

In [20]:
# train_data=train_df.copy()
# train_data=train_data.dropna()
# train_data["content"]=train_data["content"].apply(del_escape)
# train_data=train_data.dropna()
# test_data=test_df.copy()
# test_data=test_data.dropna()
# test_data["content"]=test_data["content"].apply(del_escape)
# test_data=test_data.dropna()

In [21]:
# train_user_seg_df=train_data.pivot_table(index="insta_id",values=["label"],aggfunc="median")
# train_user_seg_df=train_user_seg_df.reset_index()
# test_user_seg_df=test_data.pivot_table(index="insta_id",values=["label"],aggfunc="median")
# test_user_seg_df=test_user_seg_df.reset_index()
# train_user_text_df=train_data.pivot_table(index="insta_id",values=["content"],aggfunc="sum")
# train_user_text_df=train_user_text_df.reset_index()
# test_user_text_df=test_data.pivot_table(index="insta_id",values=["content"],aggfunc="sum")
# test_user_text_df=test_user_text_df.reset_index()

In [22]:
# X_train=np.array(train_user_text_df["content"].apply(lambda x:[int(characteristic_term[i] in x) for i in range(len(characteristic_term))]).tolist())
# y_train=np.array((train_user_seg_df["label"]==0).apply(int))
# X_test=np.array(test_user_text_df["content"].apply(lambda x:[int(characteristic_term[i] in x) for i in range(len(characteristic_term))]).tolist())
# y_test=np.array((test_user_seg_df["label"]==0).apply(int))

In [23]:
from sklearn.svm import SVC
model=SVC(probability=True)
model.fit(X,y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [24]:
test_df=pd.read_csv("community.csv")
test_df

,insta_id,content
0,163kg__,"내 주변엔 좋은사람들만 있어서 행복해막내동생이랑🤪내 코 언제 나아,,,초딩아님‼️1..."
1,97s_ba_e,마니 컸다..세탁실 앞에서 자는 그는..하루종일 잔다#사기 #보이스피싱 조심하세요...
2,again._.ty,#다시 #짧은 #글귀 #글 #시 #데일리#사진 #빛바랜 #추억 #칠하기#팔로우좋아요...
3,anyeji620,성남시 청년지원센터해피봄스데이🥳🥳.청년이봄 1주년 축하드립니다><코로나19로 인해 ...
4,clothing.wholesaler,동대문~땡!ㆍ도매언니/밤시장/ 밴드로 초대합니다.아래를 클릭하시면 밴드 가입 가능하...
5,damun_saju,"🔎후기🔎마음이 편해지는 사주, 담운사주입니다!이 시국에 나가는 게 꺼려진다면 집..."
6,dan_ggyu,김떡순 먹으러 가다가 먹고와서 일 할 생각하니 다리에 힘풀려서 길바닥 안부인사 묻고...
7,desenhos_da_rapaziada,Arrasta pro lado para ver o progresso do desen...
8,dgz_drawings,Izuku Midorya #desenhosanimes #anime #desenhos...
9,dongbo1608,대학수시(학교장추천) 가능여부로어제 오늘아침까지 전화기에 불났다불안한 맘은 알지만 ...


In [25]:
X_text=np.array(test_df["content"].apply(lambda x:[int(characteristic_term[i] in x) for i in range(len(characteristic_term))]).tolist())
X_text

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 1]])

In [26]:
pred_0=model.predict_proba(X_text)
pred_0

array([[0.69801611, 0.30198389],
       [0.82593336, 0.17406664],
       [0.78084662, 0.21915338],
       [0.54889138, 0.45110862],
       [0.67254429, 0.32745571],
       [0.86577507, 0.13422493],
       [0.88384576, 0.11615424],
       [0.76599846, 0.23400154],
       [0.74181528, 0.25818472],
       [0.81288239, 0.18711761],
       [0.72467331, 0.27532669],
       [0.84060562, 0.15939438],
       [0.71056593, 0.28943407],
       [0.64075298, 0.35924702],
       [0.6995799 , 0.3004201 ],
       [0.8020895 , 0.1979105 ],
       [0.83310397, 0.16689603],
       [0.47767832, 0.52232168],
       [0.86340622, 0.13659378],
       [0.82257268, 0.17742732],
       [0.73700001, 0.26299999],
       [0.75372274, 0.24627726],
       [0.73765144, 0.26234856],
       [0.74640988, 0.25359012],
       [0.74181528, 0.25818472],
       [0.32484559, 0.67515441],
       [0.51386738, 0.48613262],
       [0.7138032 , 0.2861968 ],
       [0.8555149 , 0.1444851 ],
       [0.13331572, 0.86668428],
       [0.

In [27]:
pred_ls=[pred_0]
pred_ls

[array([[0.69801611, 0.30198389],
        [0.82593336, 0.17406664],
        [0.78084662, 0.21915338],
        [0.54889138, 0.45110862],
        [0.67254429, 0.32745571],
        [0.86577507, 0.13422493],
        [0.88384576, 0.11615424],
        [0.76599846, 0.23400154],
        [0.74181528, 0.25818472],
        [0.81288239, 0.18711761],
        [0.72467331, 0.27532669],
        [0.84060562, 0.15939438],
        [0.71056593, 0.28943407],
        [0.64075298, 0.35924702],
        [0.6995799 , 0.3004201 ],
        [0.8020895 , 0.1979105 ],
        [0.83310397, 0.16689603],
        [0.47767832, 0.52232168],
        [0.86340622, 0.13659378],
        [0.82257268, 0.17742732],
        [0.73700001, 0.26299999],
        [0.75372274, 0.24627726],
        [0.73765144, 0.26234856],
        [0.74640988, 0.25359012],
        [0.74181528, 0.25818472],
        [0.32484559, 0.67515441],
        [0.51386738, 0.48613262],
        [0.7138032 , 0.2861968 ],
        [0.8555149 , 0.1444851 ],
        [0.133

In [28]:
# label=1,2,3 인경우 되풀이
for k in range(1,4):
    dic={}

    for term in nouns:
        try:
            reduced_term=term.replace("(","").replace(")","")
            dic[reduced_term]=[sum(data.loc[data["label"]==k]["content"].str.contains(reduced_term)),sum(data["label"]==k)-sum(data.loc[data["label"]==k]["content"].str.contains(reduced_term)),
               sum(data.loc[data["label"]!=k]["content"].str.contains(reduced_term)),sum(data["label"]!=k)-sum(data.loc[data["label"]!=k]["content"].str.contains(reduced_term))]
        except:
            print("not ordinary")
    mll_dm={}
    mll_im={}
    aic_dm={}
    aic_im={}
    relevance={}
    eps=1e-12
    for x in dic:
        mll_dm[x]=dic[x][0]*np.log(dic[x][0]+eps)+dic[x][1]*np.log(dic[x][1]+eps)+dic[x][2]*np.log(dic[x][2]+eps)+dic[x][3]*np.log(dic[x][3]+eps)-(len(data))*np.log(len(data))
    for x in dic:
        mll_im[x]=sum(data["label"]==k)*np.log(sum(data["label"]==k))+sum(data["label"]!=k)*np.log(sum(data["label"]!=k))+(dic[x][0]+dic[x][2])*np.log(dic[x][0]+dic[x][2]+eps)+(dic[x][1]+dic[x][3])*np.log(dic[x][1]+dic[x][3]+eps)-2*(len(data))*np.log(len(data))    
    for x in dic:
        aic_dm[x]=-2*mll_dm[x]+6
    for x in dic:
        aic_im[x]=-2*mll_im[x]+4
    for x in dic:
        if dic[x][0]/(dic[x][0]+dic[x][2]+eps)>dic[x][1]/(dic[x][1]+dic[x][3]+eps):
            relevance[x]=aic_im[x]-aic_dm[x]+2
        else:
            relevance[x]=aic_dm[x]-aic_im[x]-2
    characteristic_term={}
    i=0
    for x in relevance:
        if relevance[x]>0:
            characteristic_term[i]=x
            i+=1
    X=np.array(user_text_df["content"].apply(lambda x:[int(characteristic_term[i] in x) for i in range(len(characteristic_term))]).tolist())
    y=np.array((user_seg_df["label"]==k).apply(int))
    model=SVC(probability=True)
    model.fit(X,y)
    X_text=np.array(test_df["content"].apply(lambda x:[int(characteristic_term[i] in x) for i in range(len(characteristic_term))]).tolist())
    pred_ls.append(model.predict_proba(X_text))
pred_ls

not ordinary
not ordinary
not ordinary
not ordinary
not ordinary
not ordinary
not ordinary
not ordinary
not ordinary


[array([[0.69801611, 0.30198389],
        [0.82593336, 0.17406664],
        [0.78084662, 0.21915338],
        [0.54889138, 0.45110862],
        [0.67254429, 0.32745571],
        [0.86577507, 0.13422493],
        [0.88384576, 0.11615424],
        [0.76599846, 0.23400154],
        [0.74181528, 0.25818472],
        [0.81288239, 0.18711761],
        [0.72467331, 0.27532669],
        [0.84060562, 0.15939438],
        [0.71056593, 0.28943407],
        [0.64075298, 0.35924702],
        [0.6995799 , 0.3004201 ],
        [0.8020895 , 0.1979105 ],
        [0.83310397, 0.16689603],
        [0.47767832, 0.52232168],
        [0.86340622, 0.13659378],
        [0.82257268, 0.17742732],
        [0.73700001, 0.26299999],
        [0.75372274, 0.24627726],
        [0.73765144, 0.26234856],
        [0.74640988, 0.25359012],
        [0.74181528, 0.25818472],
        [0.32484559, 0.67515441],
        [0.51386738, 0.48613262],
        [0.7138032 , 0.2861968 ],
        [0.8555149 , 0.1444851 ],
        [0.133

In [29]:
# np.array(pred_ls).reshape(4,2).T[1]/np.array(pred_ls).reshape(4,2).T[1].sum()